In [1]:
import pandas as pd
import numpy as np
import os
import glob
import seaborn as sns

# 1. CẤU HÌNH
sns.set_theme(style="whitegrid")
pd.set_option('display.float_format', lambda x: '%.2f' % x)

BASE_DIR = '..'
CITY_FOLDER = 'brussels'  
PROCESSED_DIR = os.path.join(BASE_DIR, 'processed', CITY_FOLDER)

print(f"--- BẮT ĐẦU TÍNH KPI CHO: {CITY_FOLDER.upper()} ---")

# 2. HÀM TÍNH TOÁN KPI CHO TỪNG SNAPSHOT
def calculate_snapshot_kpi(snapshot_path):
    snapshot_name = os.path.basename(snapshot_path)
    print(f"\n>> Đang xử lý snapshot: {snapshot_name}")
    
    # --- Load Data ---
    try:
        df_list = pd.read_csv(os.path.join(snapshot_path, 'listings_processed.csv'), low_memory=False)
        df_cal = pd.read_csv(os.path.join(snapshot_path, 'calendar_processed.csv'), low_memory=False)
    except FileNotFoundError:
        print("   [!] Không tìm thấy file. Bỏ qua.")
        return None, None, None

    # --- Lọc dữ liệu sạch (QA Filter) ---
    # Chỉ lấy listing không bị flag lỗi giá hoặc lỗi toạ độ
    valid_listings = df_list[
        (df_list['qa_flag_price_zero'] == False) & 
        (df_list['qa_flag_out_of_city'] == False)
    ].copy()
    
    # Lọc calendar chỉ chứa các listing hợp lệ
    valid_ids = valid_listings['id'].unique()
    valid_cal = df_cal[df_cal['listing_id'].isin(valid_ids)].copy()

    # ==============================================================================
    # NHÓM 1: CUNG & HOẠT ĐỘNG
    # ==============================================================================
    # 1.1 Tổng cung
    total_supply = len(valid_listings)
    
    # 1.2 Tỷ lệ Multi-hosts (Chủ nhà có > 1 căn)
    host_counts = valid_listings['host_id'].value_counts()
    multi_host_rate = (host_counts[host_counts > 1].sum() / total_supply) * 100 if total_supply > 0 else 0

    # 1.3 Cơ cấu Room Type (Số lượng & Tỷ lệ)
    room_stats = valid_listings['room_type'].value_counts(normalize=True).reset_index()
    room_stats.columns = ['room_type', 'percentage']
    room_stats['percentage'] = room_stats['percentage'] * 100
    room_stats['snapshot_date'] = snapshot_name

    # ==============================================================================
    # NHÓM 2: GIÁ THUÊ (PRICE)
    # ==============================================================================
    # 2.1 Median Price toàn thành phố
    median_price = valid_listings['price_numeric'].median()
    
    # 2.2 Median Price theo Room Type
    price_by_room = valid_listings.groupby('room_type')['price_numeric'].median().reset_index()
    price_by_room.rename(columns={'price_numeric': 'median_price'}, inplace=True)
    price_by_room['snapshot_date'] = snapshot_name

    # ==============================================================================
    # NHÓM 3: TÍNH SẴN SÀNG (AVAILABILITY & OCCUPANCY)
    # ==============================================================================
    # 3.1 Median Availability 90 (Số ngày trống trong 3 tháng tới)
    median_avail_90 = valid_listings['availability_90'].median()
    
    # 3.2 Tỷ lệ lấp đầy (Occupancy Rate) từ Calendar
    # Logic: available='f' nghĩa là ĐÃ ĐƯỢC ĐẶT (Booked)
    valid_cal['date'] = pd.to_datetime(valid_cal['date'])
    valid_cal['is_booked'] = valid_cal['available'].apply(lambda x: 1 if x == 'f' else 0)
    
    # Tính Occupancy trung bình theo ngày (để vẽ biểu đồ mùa vụ sau này)
    daily_occupancy = valid_cal.groupby('date')['is_booked'].mean().reset_index()
    daily_occupancy.columns = ['date', 'occupancy_rate']
    daily_occupancy['snapshot_date'] = snapshot_name

    # --- Đóng gói kết quả Summary ---
    summary_row = {
        'snapshot_date': snapshot_name,
        'total_listings': total_supply,
        'multi_host_rate': multi_host_rate,
        'median_price': median_price,
        'median_avail_90': median_avail_90
    }
    
    return summary_row, room_stats, daily_occupancy

# 3. CHẠY VÒNG LẶP & TỔNG HỢP
all_snapshots = glob.glob(os.path.join(PROCESSED_DIR, '*'))
summary_list = []
room_stats_list = []
occupancy_list = []

for folder in all_snapshots:
    if os.path.isdir(folder):
        res_summary, res_room, res_occ = calculate_snapshot_kpi(folder)
        if res_summary:
            summary_list.append(res_summary)
            room_stats_list.append(res_room)
            occupancy_list.append(res_occ)

# ... (Giữ nguyên phần trên) ...

# 4. GỘP & LƯU FILE
if summary_list:
    # 4.1 Bảng Tổng hợp Chính
    df_summary = pd.DataFrame(summary_list).sort_values('snapshot_date')
    df_summary.to_csv(os.path.join(PROCESSED_DIR, f'kpi_summary_general_{CITY_FOLDER}.csv'), index=False)
    
    # 4.2 Bảng Chi tiết Room Type
    # Lọc bỏ các bảng rỗng trước khi concat để tránh Warning
    room_stats_list = [df for df in room_stats_list if not df.empty]
    if room_stats_list:
        df_room_all = pd.concat(room_stats_list, ignore_index=True)
        df_room_all.to_csv(os.path.join(PROCESSED_DIR, f'kpi_room_type_{CITY_FOLDER}.csv'), index=False)
    
    # 4.3 Bảng Seasonality (Occupancy theo ngày)
    # Lọc bỏ các bảng rỗng trước khi concat
    occupancy_list = [df for df in occupancy_list if not df.empty]
    if occupancy_list:
        df_occ_all = pd.concat(occupancy_list, ignore_index=True)
        df_occ_all.to_csv(os.path.join(PROCESSED_DIR, f'kpi_seasonality_{CITY_FOLDER}.csv'), index=False)

    print("\n--- KẾT QUẢ ---")
    print("1. Bảng tổng hợp chung (5 dòng đầu):")
    print(df_summary.head())
    print(f"\n✅ Đã lưu 3 file KPI vào: {PROCESSED_DIR}")
else:
    print("Không có dữ liệu để tính KPI.")

--- BẮT ĐẦU TÍNH KPI CHO: BRUSSELS ---

>> Đang xử lý snapshot: 16 March, 2025

>> Đang xử lý snapshot: 21 June, 2025

>> Đang xử lý snapshot: 22 December, 2024

--- KẾT QUẢ ---
1. Bảng tổng hợp chung (5 dòng đầu):
       snapshot_date  total_listings  multi_host_rate  median_price  \
0     16 March, 2025            5526            52.44         85.00   
1      21 June, 2025            5664            50.05         87.00   
2  22 December, 2024            5640            52.43         95.00   

   median_avail_90  
0            44.00  
1            49.00  
2            61.00  

✅ Đã lưu 3 file KPI vào: ..\processed\brussels
